In [1]:
#Objetivos
# patent number,
# date of patent, 
# patent title, 
# inventor name
# filed data.

In [2]:
import cv2
import re
import json
import numpy as np
from glob import glob
from pytesseract import pytesseract
import matplotlib.pyplot as plt

In [3]:
arqs = sorted(glob(r'patents/*png'))

In [4]:
class Arquive:
    def __init__(self, arq):
        self.arq = arq
        self.text = pytesseract.image_to_string(self.arq)
        self.size = len(str(self.text))
        img = cv2.imread(arq,0)
        self.shape = img.shape
        self.cut_text = pytesseract.image_to_string(img[:,:int(img.shape[1]/2)])
        
    def get_patent(self):
        patent_number = re.compile(r'(?<=Patent N).*')
        check = patent_number.findall(self.text)
        num = re.compile(r'\d')
        check_num = num.findall(str(check))
        
        return int(''.join(check_num))

    def get_date(self):
        patent_date = re.compile(r'(?<=of Patent:).*')
        check = patent_date.findall(self.text)
        date = re.compile(r'\w')
        check_date = date.findall(str(check).replace(' ','_'))
        
        return (''.join(check_date)).replace('_',' ').strip().replace('Jan','1').replace('Feb','2').replace('Mar','3').replace('Apr','4').replace('May','5').replace('Jun','6').replace('Jul','7').replace('Aug','8').replace('Sep','9').replace('Oct','10').replace('Nov','11').replace("Dec",'12').replace(' ','-')

    def get_title(self):
        patent_title = re.compile(r'(?<=54).*')
        check1 = patent_title.findall(self.cut_text)
        title = re.compile(r'[A-Z]')
        check_title = title.findall(str(check1[0]))
        if check_title == []:
            check1 = patent_title.findall(self.text)
            check_title = title.findall(str(check1[0]))
        check1 = check1[0].replace(']','').replace(')','').strip()
        patent_title2 = re.compile(f"(?<={check1}\n).*")
        check2 = (patent_title2.findall(self.cut_text))
        check=check1+' '+check2[0]
        for let in check2[0]:
            if let in '-.,[]{}()_':
                return check1.title()
        return check.title()
    #check1 para primeira linha, check2 para segunda linha e check para as duas juntas.

    def get_inventor(self):
        patent_inventor = re.compile(r'(?<=Inventor:).*')
        check = patent_inventor.findall(self.text)
        str(check).split(',')
        name = check[0].split(',')
        
        return name[0].strip()

    def get_filed(self):
        filed_date = re.compile(r'(?<=Filed).*')
        check = filed_date.findall(self.cut_text)
        date = re.compile(r'\w')
        check_date = date.findall(str(check).replace(' ','_'))
        if check_date == []:
            check = filed_date.findall(self.text)
            check_date = date.findall(str(check).replace(' ','_'))

        return (''.join(check_date)).replace('_',' ').strip().replace('Jan','1').replace('Feb','2').replace('Mar','3').replace('Apr','4').replace('May','5').replace('Jun','6').replace('Jul','7').replace('Aug','8').replace('Sep','9').replace('Oct','10').replace('Nov','11').replace("Dec",'12').replace(' ','-')

In [47]:
def get_info(arq):
    dic = dict()
    arqx = Arquive(arq)
    dic["arq"]=arqx.arq
    dic["patent_number"]=arqx.get_patent()
    dic["date_of_patent"]=arqx.get_date() 
    dic["patent_title"]=arqx.get_title() 
    dic["inventor_name"]=arqx.get_inventor()
    dic["filed_data"]=arqx.get_filed()
    
    return dic

In [ ]:
with open('patents.json', 'w'):#limpa o arquivo
    pass
    
dic_arr = list()
for arq in arqs:
    dic_arr.append(get_info(arq))
    
with open("patents.json", "a+") as outfile:#escreve no arquivo
    json.dump(dic_arr, outfile)